In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

1. Create a new spark session

In [4]:
sc.stop()

In [5]:
from pyspark import SparkConf,SparkContext
config= SparkConf().setMaster("local[2]").setAppName("SparkSession")
sc= SparkContext(conf=config)

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQLSession").getOrCreate()

In [7]:
spark


In [8]:
!hadoop fs -mkdir /airlines

mkdir: `/airlines': File exists


In [15]:
!hadoop fs -put /home/hadoop/Downloads/raw_flight_data.csv /airlines

In [16]:
!hadoop fs -put /home/hadoop/Downloads/airports.csv /airlines

put: `/airlines/airports.csv': File exists


2.Read the file and create a DF

In [9]:
#spark will infer the dataFrame with the correct datatype
flight_df  = spark.read.csv("hdfs://localhost:9000/airlines/raw_flight_data.csv",header = True , inferSchema=True)

In [11]:
flight_df.head(5)

[Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=11433, DestAirportID=13303, DepDelay=-3, ArrDelay=1),
 Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=14869, DestAirportID=12478, DepDelay=0, ArrDelay=-8),
 Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=14057, DestAirportID=14869, DepDelay=-4, ArrDelay=-15),
 Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=15016, DestAirportID=11433, DepDelay=28, ArrDelay=24),
 Row(DayofMonth=19, DayOfWeek=5, Carrier='DL', OriginAirportID=11193, DestAirportID=12892, DepDelay=-6, ArrDelay=-11)]

In [12]:
flight_df.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [13]:
flight_df.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



3. select - to select a specific column from the spark dataframe

In [14]:
flight_df.select(['DayofMonth','Carrier','OriginAirportID','DepDelay','ArrDelay']).show()

+----------+-------+---------------+--------+--------+
|DayofMonth|Carrier|OriginAirportID|DepDelay|ArrDelay|
+----------+-------+---------------+--------+--------+
|        19|     DL|          11433|      -3|       1|
|        19|     DL|          14869|       0|      -8|
|        19|     DL|          14057|      -4|     -15|
|        19|     DL|          15016|      28|      24|
|        19|     DL|          11193|      -6|     -11|
|        19|     DL|          10397|      -1|     -19|
|        19|     DL|          15016|       0|      -1|
|        19|     DL|          10397|      15|      24|
|        19|     DL|          10397|      33|      34|
|        19|     DL|          11278|     323|     322|
|        19|     DL|          14107|      -7|     -13|
|        19|     DL|          11433|      22|      41|
|        19|     DL|          11298|      40|      20|
|        19|     DL|          11433|      -2|      -7|
|        19|     DL|          10397|      71|      75|
|        1

In [15]:
flight_df['DayofMonth']

Column<b'DayofMonth'>

4. Distinct(): 
* to display the unique value from the selected columns.

In [16]:
flight_df.select(['Carrier']).distinct().show()

+-------+
|Carrier|
+-------+
|     UA|
|     AA|
|     EV|
|     B6|
|     DL|
|     OO|
|     F9|
|     YV|
|     US|
|     MQ|
|     HA|
|     AS|
|     FL|
|     VX|
|     WN|
|     9E|
+-------+



5. Columns
* to show the list of all columns from the dataFrame

In [17]:
flight_df.columns

['DayofMonth',
 'DayOfWeek',
 'Carrier',
 'OriginAirportID',
 'DestAirportID',
 'DepDelay',
 'ArrDelay']

6. count() - Data Aggregate method to show the count of records present in the DataFrame

In [18]:
flight_df.count()

2719418

In [19]:
flight_df.select(['Carrier']).distinct().count()

16

7. where()
     * it is like filter() methid if RDD with boolean conditions and statements

In [20]:
flight_df.where(flight_df.DepDelay > 10).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|        10397|     323|     322|
|        19|        5|     DL|          11433|        11298|      22|      41|
|        19|        5|     DL|          11298|        11433|      40|      20|
|        19|        5|     DL|          10397|        12451|      71|      75|
|        19|        5|     DL|          12451|        10397|      75|      57|
|        19|        5|     DL|          10397|        14771|      31|      38|
|        19|        5|     DL|          14027|      

In [21]:
flight_df.where((flight_df.DepDelay > 10)&(flight_df.ArrDelay > 20)).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|        10397|     323|     322|
|        19|        5|     DL|          11433|        11298|      22|      41|
|        19|        5|     DL|          10397|        12451|      71|      75|
|        19|        5|     DL|          12451|        10397|      75|      57|
|        19|        5|     DL|          10397|        14771|      31|      38|
|        19|        5|     DL|          14027|        10397|      13|      25|
|        19|        5|     DL|          10423|      

8. Join()
    * used to perform join operations between 2 datas

In [22]:
#read airports.csv file as sparkdf
airport_df= spark.read.options(header=True,inferSchema=True,delimiter=',').csv("hdfs://localhost:9000/airlines/airports.csv")

In [23]:
airlines_df=flight_df.join(airport_df,flight_df.OriginAirportID==airport_df.airport_id)

In [24]:
airlines_df.show()

+----------+---------+-------+---------------+-------------+--------+--------+----------+-----------------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|             city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+-----------------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|          Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|   Salt Lake City|   UT|Salt Lake City In...|
|        19|        5|     DL|          14057|        14869|      -4|     -15|     14057|         Portland|   OR|Portland Internat...|
|        19|        5|     DL|          15016|        11433|      28|      24|     15016|        St. Louis|   MO|Lambert-St. Louis...|
|        19|        5|     DL|          11193|        1

In [25]:
airlines_df_left = flight_df.join(airport_df, flight_df.OriginAirportID == airport_df.airport_id, how = 'left_outer')

In [26]:
airlines_df_left.show(5)

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|          city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|       Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake City|   UT|Salt Lake City In...|
|        19|        5|     DL|          14057|        14869|      -4|     -15|     14057|      Portland|   OR|Portland Internat...|
|        19|        5|     DL|          15016|        11433|      28|      24|     15016|     St. Louis|   MO|Lambert-St. Louis...|
|        19|        5|     DL|          11193|        12892|      -6|     -1

9. isnull()
    * DataFrame method which returns boolean output for misiing values

In [27]:
# Check for null value for each specific column name
from pyspark.sql.functions import col,when,isnull,count

flight_df.filter(col('ArrDelay').isNull()).count()

29033

In [28]:
flight_df.filter(col('DepDelay').isNull()).count()

27444

In [29]:
flight_df.select([count(when(isnull(column), column)).alias(column) for column in flight_df.columns]).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         0|        0|      0|              0|            0|   27444|   29033|
+----------+---------+-------+---------------+-------------+--------+--------+



10. fillna()
    * Replace or fill the missing values of column with central tendencies such as mean , median or mode
    * Replace missing value by 0

In [30]:
flight_df1=flight_df.fillna({'DepDelay':0,'ArrDelay':0})

In [31]:
flight_df1.select([count(when(isnull(column),column)).alias(column) for column in flight_df1.columns]).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         0|        0|      0|              0|            0|       0|       0|
+----------+---------+-------+---------------+-------------+--------+--------+



11. dropna()
    * drop all missing rows that are having null values

In [32]:
flight_df2 = flight_df.dropna()

12. Summary of the data

In [34]:
flight_df2.summary().show()

+-------+------------------+------------------+-------+------------------+------------------+------------------+-----------------+
|summary|        DayofMonth|         DayOfWeek|Carrier|   OriginAirportID|     DestAirportID|          DepDelay|         ArrDelay|
+-------+------------------+------------------+-------+------------------+------------------+------------------+-----------------+
|  count|           2690385|           2690385|2690385|           2690385|           2690385|           2690385|          2690385|
|   mean|15.797197427134035|   3.9010870191441|   null|12741.989083718501| 12742.32015157682|10.512613250519907| 6.63768791455498|
| stddev| 8.806016759236407|1.9878641519635314|   null|1502.7991156844841|1502.9075179984509|36.033091528336584|38.64881489390021|
|    min|                 1|                 1|     9E|             10140|             10140|               -63|              -94|
|    25%|                 8|                 2|   null|             11292|         

In [35]:
!pip install numpy pandas matplotlib seaborn


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached pandas-1.1.5-cp36-cp36m-manylinux1_x86_64.whl (9.5 MB)
     |████████████████████████████████| 11.5 MB 8.7 MB/s            
     |████████████████████████████████| 292 kB 50.1 MB/s            
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
     |████████████████████████████████| 1.1 MB 9.1 MB/s            
     |████████████████████████████████| 3.1 MB 27.7 MB/s            :00:01
     |████████████████████████████████| 25.9 MB 2.2 MB/s             


In [38]:
flight_df2.summary().toPandas()

,summary,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay
0,count,2690385,2690385,2690385,2690385,2690385,2690385,2690385
1,mean,15.797197427134035,3.9010870191441,None,12741.989083718501,12742.32015157682,10.512613250519907,6.63768791455498
2,stddev,8.806016759236407,1.9878641519635314,None,1502.7991156844841,1502.9075179984509,36.033091528336584,38.64881489390021
3,min,1,1,9E,10140,10140,-63,-94
4,25%,8,2,None,11292,11292,-4,-11
5,50%,16,4,None,12892,12892,-1,-3
6,75%,23,6,None,14057,14057,9,10
7,max,31,7,YV,15376,15376,1863,1845


13. Describe 
    * Use describe function to show brief summary statistics for dataframe.
    summary includes count(), stddev() ,min(),max(),mean()of the data

In [40]:
flight_df2.describe().toPandas()

,summary,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay
0,count,2690385,2690385,2690385,2690385,2690385,2690385,2690385
1,mean,15.797197427134035,3.9010870191441,None,12741.989083718501,12742.32015157682,10.512613250519907,6.63768791455498
2,stddev,8.806016759236407,1.9878641519635314,None,1502.7991156844841,1502.9075179984509,36.033091528336584,38.64881489390021
3,min,1,1,9E,10140,10140,-63,-94
4,max,31,7,YV,15376,15376,1863,1845


14. col.()groupBy()
    * col() - method used to refer to a column in the dataframe
    * groupBy() - method used to group the row of DF based on column values

In [42]:
from pyspark.sql.functions import col
flight_df2.filter(col("ArrDelay")>100).count()

81354

In [43]:
flight_df2.groupBy(col("carrier")).count().show()

+-------+------+
|carrier| count|
+-------+------+
|     UA|285207|
|     AA|288118|
|     EV|153808|
|     B6|121736|
|     DL|384325|
|     OO|158424|
|     F9| 35761|
|     YV| 51803|
|     US|233045|
|     MQ|109307|
|     HA| 18633|
|     AS| 68900|
|     FL| 92393|
|     VX| 34794|
|     WN|576220|
|     9E| 77911|
+-------+------+



15. with column
    * all column operations - used to create a new column in the existing DF
    * modifying an existing column in the DF
    

In [45]:
flight_df3 =flight_df2.withColumn("TotalDelay", col("DepDelay")+col("ArrDelay"))

In [46]:
flight_df3.columns

['DayofMonth',
 'DayOfWeek',
 'Carrier',
 'OriginAirportID',
 'DestAirportID',
 'DepDelay',
 'ArrDelay',
 'TotalDelay']

In [49]:
flight_df4 = flight_df3.withColumn( "isDelay",when(col("DepDelay") > 0, "Delayed").otherwise("on-time"))

In [50]:
flight_df4.show()

+----------+---------+-------+---------------+-------------+--------+--------+----------+-------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|TotalDelay|isDelay|
+----------+---------+-------+---------------+-------------+--------+--------+----------+-------+
|        19|        5|     DL|          11433|        13303|      -3|       1|        -2|on-time|
|        19|        5|     DL|          14869|        12478|       0|      -8|        -8|on-time|
|        19|        5|     DL|          14057|        14869|      -4|     -15|       -19|on-time|
|        19|        5|     DL|          15016|        11433|      28|      24|        52|Delayed|
|        19|        5|     DL|          11193|        12892|      -6|     -11|       -17|on-time|
|        19|        5|     DL|          10397|        15016|      -1|     -19|       -20|on-time|
|        19|        5|     DL|          15016|        10397|       0|      -1|        -1|on-time|
|        19|        

16. Statistical Method

In [62]:
from pyspark.sql.functions import mean, stddev,variance,round


In [65]:
flight_df4.select(round(mean(col('DepDelay'))).alias('DepDelay_mean')).show()

+-------------+
|DepDelay_mean|
+-------------+
|         11.0|
+-------------+



In [69]:
flight_df4.select(round(variance(col('DepDelay'))).alias('DepDelay_variance')).show()

+-----------------+
|DepDelay_variance|
+-----------------+
|           1298.0|
+-----------------+



In [70]:
flight_df4.select(round(stddev(col('DepDelay'))).alias('DepDelay_std_dev')).show()

+----------------+
|DepDelay_std_dev|
+----------------+
|            36.0|
+----------------+



In [71]:
flight_df4.approxQuantile("DepDelay",[0.5],0.01)[0]

-1.0